<hr/>

# Introduction to Data Science - Fall 2021
**Jinchao Feng and Christian Kuemmerle** - introdsfall2021@jh.edu <br/>


- Ridge Regression & Sparse Regression

- Linear Algebra Recap
<hr/>

## Recap: Linear Regression/Ordinary Least Squares

(see written notes)

# Regularization: Making Linear Regression more flexible

Penalize large coefficients in $\beta$

- **Ridge regression** uses $L_2$-regularization, where $\|\beta\|^2_2 = \sum_{i=1}^N \beta_i^2$

> $\displaystyle \hat{\beta} = \arg\!\min_{\beta} \, \| y- X\beta \,\|^2_2\ + \alpha\,\|\beta\|^2_2$  
><br/>
> or even with a constant matrix $\Gamma$
><br/><br/>
> $\displaystyle \hat{\beta} = \arg\!\min_{\beta} \, \| y- X\beta \,\|^2_2 + \alpha\,\|\Gamma\beta\|^2_2$  

- **Lasso regression** uses $L_1$-regularization, where $\|\beta\|_1 = \sum_{i=1}^N |\beta_i|$.

> $\displaystyle \hat{\beta} = \arg\!\min_{\beta}  \, \| y- X\beta \,\|^2_2 + \alpha\,\|\beta\|_1$ 
><br/><br/>
> $L_1$ yields sparse results

Different geometric meanings! See written notes for more details.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

We define a function to sample $n=\text{'n_sample'}$ points $x_1,\ldots,x_n$ uniformly distributed at random and define 
corresponding samples $y_i=f(x_i) + \epsilon_i$ where 
$f(x)= \sin(4 x) + x$ and $\epsilon_i$ are i.i.d. standard random variables. 

In [ ]:
### function to create random samples from a wave-like signal
def make_wave(n_samples=100):
    rnd = np.random.RandomState(42)
    x = rnd.uniform(-3, 3, size=n_samples)
    x = np.sort(x)
    y_no_noise = (np.sin(4 * x) + x)
    y = y_no_noise + rnd.normal(size=len(x))/2
    return x.reshape(-1, 1), y

# Visualize the function f:
fn = lambda x: np.sin(4 * x) + x
line_xvalues = np.linspace(-3, 3, 1000).reshape(-1, 1)
plt.figure(figsize=(14,8))
plt.plot(line_xvalues,fn(line_xvalues))
ax = plt.gca()
ax.set_ylim(-6, 6)
ax.set_xlim(-4, 4)
ax.grid(True)

We create a data set (consisting of $x$-coordinates in '$X$' and $y$-coordinates in '$y$') based on the above model, using a 'nr_samples' samples.

In [ ]:
# %% Create data, create training and test data split
nr_samples = 30
X, y = make_wave(n_samples=nr_samples)
y = y.reshape((len(y), 1))
dfX = pd.DataFrame(X)

We now use the method [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train_test#sklearn.model_selection.train_test_split) of scikit-learn to split the available samples into to groups, a training dataset and a test dataset. The share of samples in the training data is provided by the choice of 'train_share', i.e., if $\text{train_share} = 0.6$, then 60% of the samples are used in the training dataset.

From the samples variable $X$, we create a [pandas](https://pandas.pydata.org/docs/) DataFrame as its methods are sometimes more convenient to use - for example, we can easily obtain a vector of indices corresponding to the training and test set, respectively. We define the corresponding vectors 'id_train' and 'id_test'.

In [ ]:
#train_share = 0.6
#dfX = pd.DataFrame(X)
#X_train, X_test, y_train, y_test = train_test_split(dfX, y, train_size=train_share,random_state=25) # split data into  a share of 'train_share' in training data set and  a share of 1-'train_share' in test data set
#id_train = X_train.index # obtain index of training set
#X_train = X_train.to_numpy()
#id_test = X_test.index # obtain index of test set
#X_test= X_test.to_numpy()

The samples can be visualized as follows.

In [ ]:
# %% Plot training data
plt.figure(figsize=(14,8))
plt.plot(X,y,'o',c='blue')
ax = plt.gca()
ax.set_ylim(-6, 6)
ax.set_xlim(-4, 4)
ax.spines['left'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('center')
ax.spines['top'].set_color('none')
ax.legend(["data points"],loc=0,fontsize=8)
ax.grid(True)

We now train a linear regression model (with $\alpha=\text{alphaval}$) on the samples of the training set, and evaluate the mean squared errors as well as the resulting $R^2$ coefficient. We also train a linear and a ridge regression model on **polynomial features** of degree='degree_poly'.

We fit first the linear regression model to the training data with unmodified features. This corresponds to a truly linear model in the standard coordinate system.

In [ ]:
# %% Fit models to training data
lr = LinearRegression().fit(X, y) # linear regression

Use polynomial features

In [ ]:
# %% Run linear regression with polynomial features of degree 'degree_poly':
from sklearn.preprocessing import PolynomialFeatures
degree_poly = 24;
poly = PolynomialFeatures(degree_poly) # define polynomial data model
X_poly = poly.fit_transform(X) # obtain coordinates of polynomial features of training set
polylr = LinearRegression().fit(X_poly,y) # perform linear regression with polynomial features

Now, we use ridge regression on the polynomial features for a fixed regularization parameter $\alpha=\text{alphaval}$. Please note the options of [linear_models](https://scikit-learn.org/stable/modules/linear_model.html#linear-model).

In [ ]:
import warnings
from scipy.linalg import LinAlgWarning
warnings.filterwarnings("ignore", category=LinAlgWarning)
# %% Run ridge regression with polynomial features
alphaval = 1 # ridge regression parameter alpha
polyridge = Ridge(alpha=alphaval).fit(X_poly,y) # perform ridge regression with polynomial features
alphaval2 = 10**3 # ridge regression parameter alpha
polyridge2 = Ridge(alpha=alphaval2).fit(X_poly,y) # perform ridge regression with polynomial features
line = np.linspace(-8, 8, 1000).reshape(-1, 1)
line_poly = poly.transform(line)

We output the $R^2$ for the three considered models, for both training and test set, as well as the _mean squared errors_ for these.

In [ ]:
# %% Plot summary
plt.figure(figsize=(14,8))
plt.plot(X,y,'o',c='blue')
#plt.plot(line, lr.predict(line))
plt.plot(line, polylr.predict(line_poly))
plt.plot(line, polyridge.predict(line_poly))
plt.plot(line, polyridge2.predict(line_poly))
ax = plt.gca()
ax.spines['left'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('center')
ax.spines['top'].set_color('none')
ax.set_ylim(-6, 6)
ax.set_xlim(-4, 4)
ax.legend(["data points","OLS (Ridge with $alpha = 0$)","Ridge with alpha="+str(alphaval),"Ridge with alpha="+str(alphaval2)], loc=0,fontsize=10)
ax.grid(True)

We observe that for larger regularization parameters $\alpha$, the regression function is "smoother". From the perspective of the regression coefficients (cooridnates of the vector $\widehat{\beta}$), we see that for large $\alpha$, the coefficients are "shrinked" by Ridge regression.

In [ ]:
plt.figure()
plt.plot(polylr.coef_.T, '^', label="OLS (Ridge with $alpha = 0$)")
plt.plot(polyridge.coef_.T, 'v', label="Ridge with alpha="+str(alphaval))
plt.plot(polyridge2.coef_.T, 'x', label="Ridge with alpha="+str(alphaval2))
plt.legend(ncol=2, loc=(0, 1.05))
#plt.ylim(-25, 25)
plt.xlabel("Coefficient index")
plt.ylabel("Coefficient magnitude")
plt.show()

## Exploring the Bias-Variance Tradeoff

- Bias: the difference between the average prediction of our model and the correct value which we are trying to predict. 
> Model with high bias pays very little attention to the training data and oversimplifies the model. It always leads to high error on training and test data (underfitting).

- Variance: the variability of model prediction for a given data point or a value which tells us the spread of our data.
> Model with high variance pays a lot of attention to training data and does not generalize on the data which it hasn’t seen before. As a result, such models perform very well on training data but has high error rates on test data (overfitting).

<img src='files/BiasVarianceTradeoff.png' width=400 align=left>

### Ridge Regression

We recall the [diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) dataset from last week's problem set. We use `StandardScaler` from scikit-learn to standardize the scale of the dataset.

In [ ]:
import pandas as pd
# Download the diabetes file, and save it in the working directory
# Read the text file into a pandas dataframe
diabetes = pd.read_csv("https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt", sep='\t')
#diabetes

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
diabetes_df = diabetes.drop(columns = ['SEX'])
diabetes_df
scaler = StandardScaler()
saved_cols = diabetes_df.columns
diabetes_scaled = scaler.fit_transform(diabetes_df)
diabetes_df_scaled = pd.DataFrame(diabetes_scaled, columns = saved_cols)
#diabetes_df_scaled

We use a seperate the dataset at random into a training set that contains 80% of the samples and a test set that contains 20% of the samples. 

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(diabetes_df_scaled, test_size=0.2)
train_X = train.drop(columns = ["Y"])
train_y = train["Y"]
test_X = test.drop(columns = ["Y"])
test_y = test["Y"]
#test_X

We now train [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html?highlight=ridge#sklearn.linear_model.Ridge) regression models for different regularization parameters $\alpha$.
For this, we first define a function `computeScoresMSE` that computes R2 scores and MSE values of an estimator for a range of regularization parameters.

In [ ]:
def computeScoresMSE(model,alphas,train_X,train_y,test_X,test_y):
    train_scores = np.empty_like(alphas)
    test_scores = np.empty_like(alphas)
    test_MSE    = np.empty_like(alphas)
    train_MSE   = np.empty_like(alphas)
    models = []
    for i in range(alphas.size):
        models.append(model(alpha=alphas[i]))
        models[i].fit(train_X,train_y)
        train_scores[i] = models[i].score(train_X, train_y)
        test_scores[i]  = models[i].score(test_X, test_y)
        test_MSE[i]     = np.mean((test_y-models[i].predict(test_X))**2)
        train_MSE[i]    = np.mean((train_y-models[i].predict(train_X))**2)
    index = np.argmin(test_MSE)
    best_model = models[index]
    return train_MSE, test_MSE, train_scores, test_scores, best_model

In [ ]:
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
alphas = np.logspace(-2,6,num=60) # create numpy array of logarithmically interpolated values between 10^(-5) and 10^(9)
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(1,interaction_only=False)
poly.fit(train_X)
features_names = poly.get_feature_names(train_X.columns)

In [ ]:
p_train_X = poly.transform(train_X)
p_test_X = poly.transform(test_X)

train_MSE, test_MSE, train_scores, test_scores, Ridge_opt = \
    computeScoresMSE(Ridge,alphas,p_train_X,train_y,p_test_X,test_y)
plt.figure(figsize=(14,5))
plt.semilogx(alphas,test_MSE,'x',label="Test MSE")
plt.semilogx(alphas,train_MSE,'x',label="Train MSE")
plt.xlabel(r"Regularization parameter $\alpha$")
plt.ylabel(r"Mean Squared Error (MSE)")
plt.title("Bias-Variance Tradeoff for Ridge Regression")
ax = plt.gca()
ax.invert_xaxis()
plt.legend()
plt.show()

print("Model with optimal regularization parameter alpha: ",Ridge_opt)
print("Best test MSE: ",min(test_MSE)) # compare with np.mean((test_y-Ridge_opt.predict(test_X))**2)
(min(test_MSE) , np.mean((test_y-Ridge_opt.predict(p_test_X))**2))

In [ ]:
max(test_scores)
y_test_avg = np.mean(test_y)
Ridge

In [ ]:
rsquared = 1 - np.linalg.norm(test_y-Ridge_opt.predict(p_test_X))**2 / np.linalg.norm(test_y-y_test_avg)**2
(max(test_scores),rsquared)

### Lasso Regression
We repeat this experiment for the Lasso.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning) # this is to suppress warnings related to the optimization in the training of Lasso

train_MSE, test_MSE, train_scores, test_scores, Lasso_opt = \
    computeScoresMSE(Lasso,alphas,p_train_X,train_y,p_test_X,test_y)
plt.figure(figsize=(14,5))
plt.semilogx(alphas,test_MSE,'x',label="Test MSE")
plt.semilogx(alphas,train_MSE,'x',label="Train MSE")
plt.xlabel(r"Regularization parameter $\alpha$")
plt.ylabel(r"Mean Squared Error (MSE)")
plt.title("Bias-Variance Tradeoff for Lasso Regression")
ax = plt.gca()
ax.invert_xaxis()
plt.legend()
plt.show()

print("Model with optimal regularization parameter alpha: ",Lasso_opt)
print("Best test MSE: ",np.mean((test_y-Lasso_opt.predict(p_test_X))**2))

We observe that the regularization parameter $\alpha$ resulting in the smallest test MSE is different for Ridge and Lasso regression.

Finally, we create a plot that visualizes the differences between the coefficients returned by the two different methods.

In [ ]:
Lasso_opt.coef_

In [ ]:
Ridge_opt.coef_

In [ ]:
plt.figure()
plt.plot(Lasso_opt.coef_, '^', label=str(Lasso_opt))
plt.plot(Ridge_opt.coef_, 'v', label=str(Ridge_opt))
plt.plot(np.arange(np.size(Lasso_opt.coef_)),np.zeros(np.size(Lasso_opt.coef_)))
plt.legend(ncol=2, loc=(0, 1.05))
#plt.ylim(-25, 25)
plt.xlabel("Coefficient index")
plt.ylabel("Coefficient magnitude")
plt.xticks(ticks = np.arange(Lasso_opt.coef_.size),labels=features_names)
plt.show()
#ax = plt.gca()
#ax.set_xlabels(features_names)

The coefficient vector $\widehat{\beta}$ returned by Lasso is _sparser_ then the one returned by Ridge regression. This leads to a better interpretability of the results Lasso, as it performs _variable selection_: Ideally, we can infer from the non-zero coeffcients of the coefficient vector $\widehat{\beta}$ _which predictor variables_ are most relevant.

#### Regularized Regression & Bias-Variance Tradeoff
For both Ridge regression and Lasso regression it is true that:
  - $\alpha \approx 0$ or small: Larger variance, smaller bias
  - $\alpha$ large: Small variance, large bias

# Linear Algebra for Data Science

Matrix decompositions and linear algebra are at the core of many techniques in data science.
Please review Chapter 4 (and, if necessary, Chapter 2) of the [book by Deisenroth, Faisal and Ong](https://mml-book.github.io/book/mml-book.pdf).
### Data sample matrix and sample covariance matrix

- We generate $n = 20$ points in 2d sampled as i.i.d. standard Gaussians. Their $x$-coordinates are streched by a factor $4$, and their location is subsequently rotated by $45°$, after which a shift by $(1,3)$ is applied. This gives rise to the data matrix $X \in \mathbb{R}^{2 \times 20}$.

In [ ]:
%pylab inline

In [ ]:
from scipy.stats import norm
n = 20
# generate multiple 2-D (column) vectors
S = norm.rvs(0,1,(2,n))
S[0,:] *= 4  # scale axis 0
f = +pi/4    # rotate by 45 degrees
R = array([[cos(f), -sin(f)],
           [sin(f),  cos(f)]]) 
X = R.dot(S)
X += np.array([[1],[3]]) # shift

figure(figsize=(5,5)); xlim(-15,15); ylim(-15,15);
plot(X[0,:],X[1,:],'o',alpha=0.9)

Let $\overline{X}$ be the matrix with row averages of $X$, and $\widetilde{X} = X -\overline{X}$.
> Sample covariance matrix corresponds to
><br/><br/>
>$\displaystyle C =  \frac{1}{n\!-\!1}\widetilde{X}\widetilde{X}^T =  \frac{1}{n\!-\!1}\ (X- \overline{X})(X- \overline{X})^T = \frac{1}{n\!-\!1}\  \sum_{i=1}^n (x_i- \overline{x}) (x_i- \overline{x})^T$

We observe that there is a close relationship between the

- Singular Value Decomposition (SVD)

>$\displaystyle \widetilde{X} = U W V^T$
><br/><br/>
> where $U^TU=I$, $W$ is diagonal, and $V^TV=I$,
>> Columns of $U$ are **left singular vectors** of $\widetilde{X}$, <br>
>> Columns of $V$ are **right singular vectors** of $\widetilde{X}$.

and the 
- Eigendecomposition

>$\displaystyle C=E\Lambda E^T$ <br><br>
> of the sample covariance matrix $C$, since

>$\displaystyle C = \frac{1}{n\!-\!1}\  UWV^T\ VWU^T = \frac{1}{n\!-\!1}\ U W^2 U^T$.
><br/><br/>
> So, if $C=E\Lambda E^T$ then $E = U$ and $\displaystyle \Lambda = \frac{1}{n\!-\!1}\  W^2$.

In [ ]:
X

In [ ]:
# subtract sample mean
avg = mean(X, axis=1).reshape(X[:,1].size,1)
Xtilde = X- avg
Xtilde

In [ ]:
# sample covariance matrix
C = Xtilde.dot(Xtilde.T) / (X[0,:].size-1) 
print ("Average\n", avg)
print ("Covariance\n", C)

In [ ]:
L, E = np.linalg.eig(C)
E, L

In [ ]:
E, L, E_same = np.linalg.svd(C)
E, L

In [ ]:
E.dot(E.T)

In [ ]:
np.allclose( E.T, np.linalg.inv(E) )

In [ ]:
U, W, V = np.linalg.svd(X)
U, W**2 / (X[0,:].size-1)

In [ ]:
# alternatively
U, W**2 / (X.shape[1]-1)

In [ ]:
[ np.allclose( U.dot(U.T), np.eye(U.shape[0]) ), 
  np.allclose( V.dot(V.T), np.eye(V.shape[0]) )  ]